In [20]:
import warnings

# Ignore all warnings
warnings.filterwarnings('ignore')


In [21]:
import pandas as pd
from mlforecast import MLForecast
from xgboost import XGBRegressor
from utilsforecast.plotting import plot_series
from matplotlib import pyplot as plt
from window_ops.rolling import rolling_mean, rolling_std, rolling_min, rolling_max
from mlforecast.target_transforms import Differences
from sklearn.metrics import mean_squared_error, accuracy_score
import numpy as np
from scipy import signal
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from prophet import Prophet
from sklearn.base import BaseEstimator, RegressorMixin
from statsmodels.tsa.seasonal import STL
from lightgbm import LGBMRegressor

In [22]:
df=pd.read_csv('MLTempDataset.csv')
df.drop(columns=['Unnamed: 0',],inplace=True)#'Datetime1'
df1=df.copy()
df1['unique_id']='h1'
# df1['unique_id']=
df1['ds']=pd.to_datetime(df['Datetime'])
df1['y']=df['DAYTON_MW']
df1.drop(columns=['Datetime','DAYTON_MW','Datetime1'],inplace=True)
df1.head()

,unique_id,ds,y
0,h1,2022-01-04 00:00:00,20.867
1,h1,2022-01-04 01:00:00,21.000
2,h1,2022-01-04 02:00:00,20.867
3,h1,2022-01-04 03:00:00,20.650
4,h1,2022-01-04 04:00:00,20.400


In [23]:
trainbef=df1[:5340]
testbef=df1[5340:]
trainbef.shape,testbef.shape

((5340, 3), (1336, 3))

In [24]:
from scipy.signal import periodogram

frequencies, spectrum = periodogram(df1['y'])
max_idx = np.argmax(spectrum)
second_max_idx = np.argsort(spectrum)[-2]  # Get the second largest index

# Get the maximum and second maximum values
max_value = spectrum[max_idx]
second_max_value = spectrum[second_max_idx]

# Get the corresponding frequencies
max_freq = frequencies[max_idx]
second_max_freq = frequencies[second_max_idx]

# plt.figure(figsize=(14, 7))
# plt.plot(frequencies, spectrum)
# plt.title('Periodogram')
# plt.xlabel('Frequency')
# plt.ylabel('Power')
# plt.show()

In [25]:
1/max_freq, 1/second_max_freq

(np.float64(3338.0), np.float64(667.6))

In [26]:
seasonal_period=667

In [27]:
stl = STL(df1['y'],period=seasonal_period)
result=stl.fit()
# result.plot()

In [28]:
trend=result.trend
trend.head()
seasonal=result.seasonal

In [29]:
a=pd.DataFrame(trend)
a.shape

(6676, 1)

In [30]:
a['ds']=df1['ds']
a['unique_id']=df1['unique_id']
a['y']=a['trend']

In [31]:
a=a.dropna().reset_index(drop=True)
a.head()

,trend,ds,unique_id,y
0,19.671413,2022-01-04 00:00:00,h1,19.671413
1,19.667384,2022-01-04 01:00:00,h1,19.667384
2,19.663358,2022-01-04 02:00:00,h1,19.663358
3,19.659334,2022-01-04 03:00:00,h1,19.659334
4,19.655312,2022-01-04 04:00:00,h1,19.655312


In [32]:
a.shape

(6676, 4)

In [ ]:
# mlf = MLForecast(
#     models=[LGBMRegressor(), XGBRegressor(), LinearRegression()],  # List of models for forecasting: LightGBM, XGBoost and Linear Regression
#     freq='D',  # Frequency of the data - 'D' for daily frequency
#     lags=list(range(1, 7)),  # Specific lags to use as regressors: 1 to 6 days
#     lag_transforms = {
#         1:  [expanding_mean],  # Apply expanding mean transformation to the lag of 1 day
#     },
#     date_features=['year', 'month', 'day', 'dayofweek', 'quarter', 'week'],  # Date features to use as regressors
# )

# 'boosting_type': 'gbdt',
#     'num_leaves': 31,
#     'learning_rate': 0.1,
#     'n_estimators': 100,

In [35]:
model=XGBRegressor(random_state=990,learning_rate=0.01,n_estimators=500,max_depth=10,reg_lambda=0.2)
model2=LGBMRegressor(boosting_type='gbdt',num_leaves=31,learning_rate=0.1,n_estimators=100,random_state=990)
ftsc=MLForecast(
    models=[model,model2],
    freq='H',
    lags=[seasonal_period],
    lag_transforms={
        # 30:[(rolling_mean,30),(rolling_std,30),(rolling_min,30),(rolling_max,30)],
        # 168:[(rolling_mean,168),(rolling_std,168),(rolling_min,168),(rolling_max,168)],
        seasonal_period:[(rolling_mean,seasonal_period),(rolling_std,seasonal_period),(rolling_min,seasonal_period),(rolling_max,seasonal_period)],
    #     350:[(rolling_mean,350),(rolling_std,350),(rolling_min,350),(rolling_max,350)],
    #     720:[(rolling_mean,720),(rolling_std,720),(rolling_min,720),(rolling_max,720)]
    },
    # target_transforms=[Differences([350])],
    date_features=['hour','day','weekday','month','year'],
)
# ftsc.fit(train,id_col='unique_id',time_col='ds',target_col='y')
# min_child_weight=
# ,gamma=
# ,subsample=
# objective="reg:squarederror",

In [36]:
crossvalidation_df = ftsc.cross_validation(
    df=a,
    h=168,
    n_windows=5,
)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001245 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1602
[LightGBM] [Info] Number of data points in the train set: 4503, number of used features: 10
[LightGBM] [Info] Start training from score 22.088450
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000182 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1602
[LightGBM] [Info] Number of data points in the train set: 4671, number of used features: 10
[LightGBM] [Info] Start training from score 22.143811
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003550 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bi

In [40]:
crossvalidation_df.head()

,unique_id,ds,cutoff,y,XGBRegressor,LGBMRegressor
0,h1,2022-09-04 04:00:00,2022-09-04 03:00:00,23.903974,23.893799,23.910899
1,h1,2022-09-04 05:00:00,2022-09-04 03:00:00,23.900103,23.893799,23.910899
2,h1,2022-09-04 06:00:00,2022-09-04 03:00:00,23.896237,23.893799,23.910899
3,h1,2022-09-04 07:00:00,2022-09-04 03:00:00,23.892379,23.893799,23.910899
4,h1,2022-09-04 08:00:00,2022-09-04 03:00:00,23.888527,23.893799,23.910899


In [42]:
from utilsforecast.losses import rmse

crossvalidation_df['id_cutoff'] = crossvalidation_df['unique_id'] + '_' + crossvalidation_df['cutoff'].astype(str)
cv_rmse = rmse(crossvalidation_df, models=['LGBMRegressor', 'XGBRegressor'], id_col='id_cutoff')
# print("RMSE using cross-validation: ", cv_rmse.mean())

In [44]:
cv_rmse['LGBMRegressor'].mean(),cv_rmse['XGBRegressor'].mean()

(np.float64(0.11889292339133775), np.float64(0.11330350801123576))